In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

# On va chercher les données dans le fichier CSV
data = pd.read_csv('./data/train.csv')


# On va séparer les données en deux parties : les commentaires et les labels
X = data['comment_text']
y = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# On va séparer les données en deux parties : une pour l'entraînement et une pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# On définit les paramètres, max_words est le nombre de mots maximum que l'on va garder, max_len est la longueur maximale d'un commentaire.
max_words = 20000
max_len = 150

# Ici le tokenizer va transformer les commentaires en séquences de nombres.
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# On va ajouter des 0 pour que toutes les séquences aient la même longueur.
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# On va entraîner un modèle pour chaque label ce qui va nous permettre d'avoir un modèle pour chaque type de commentaire (plus performant que d'avoir un seul modèle pour tous les labels).
for label in y.columns:
    print(f"Training model for {label}")

    # On va utiliser un RandomUnderSampler pour équilibrer les classes, càd que l'on va supprimer des exemples de la classe majoritaire (0) pour qu'il y ait autant d'exemples dans la classe minoritaire (1).
    rus = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X_train_pad, y_train[label])

    # On crée un modèle avec une couche d'embedding, une couche de LSTM et une couche dense.
    model = Sequential()
    model.add(Embedding(max_words, 100, input_length=max_len))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # On compile le modèle avec une loss binary_crossentropy, un optimizer adam et une métrique accuracy.
    # On utilise binary_crossentropy car on a un problème de classification binaire, en gros on veut prédire si un commentaire est toxique ou non.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    history = model.fit(X_resampled, y_resampled, epochs=10, batch_size=64, validation_split=0.1)

    y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")

    report = classification_report(y_test[label], y_pred, labels=[0, 1])
    print(report)

    # On va afficher la matrice de confusion pour chaque label.
    cm = confusion_matrix(y_test[label], y_pred)
    TN, FP, FN, TP = cm.ravel()

    # En plus, le F1 Score.
    f1 = f1_score(y_test[label], y_pred)
    print(f"F1 Score: {f1:.4f}")

    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
    plt.title(f'Confusion Matrix for {label}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
